In [1]:
import os
import pandas as pd

workdir = os.path.abspath('./')
print(workdir)

/Users/dongjiacheng/Desktop/code/lab/YM


In [2]:
# pd.读取excel
df_sc = pd.read_excel('./autophagy gene list-yeast-total 268 gene.xlsx')
df_sc.head()

,Gene,Ano_seq
0,YDR425W,MDYNIFEAVHEQQSSTSDMDLSEEDNNPFVGTHHLYASGIGTTIGE...
1,YJL036W,MTDKGKNDLTSKAKDKARGNPEKPPYWFEIIVSDPQKRTGDPGSSS...
2,YEL013W,MGSCCSCLKDSSDEASVSPIADNEREAVTLLLGYLEDKDQLDFYSG...
3,YHR195W,MTRPPLVRGIFSLGLSVAVLKGVEKTVRKHLERQGWIEPQKVDYEL...
4,YKR068C,MVSTTQSRSLKAMGEEIWKNKTEKINTELFTLTYGSIVAQLCQDYE...


In [5]:
"""
将df_sc中的数据，保存到一个txt中，要求格式如下：
>Protein_id + Gene + Symbol
Amin_seq
.........
"""
# 将df_sc中的数据，保存到一个txt中
with open('blast_input_Sc.txt', 'w') as f:
    for i in range(len(df_sc)):
        f.write('>' + df_sc['Gene'][i] + '\n')
        f.write(df_sc['Ano_seq'][i] + '\n')

In [6]:
# 在jupyter中添加环境变量
os.environ['PATH'] += os.pathsep + '/Users/dongjiacheng/Desktop/code/mtd/tools/blast/bin'

In [7]:
os.system('blastp -query blast_input_Sc.txt -out blast_output_Sc.txt -db /Users/dongjiacheng/Desktop/code/mtd/tools/blast/db_prot/Mt-Nc -outfmt 6 -evalue 1e-6')

0

In [8]:
# 读取blast_output_Sc.txt,
df_blast = pd.read_csv('blast_output_Sc.txt', sep='\t', header=None).copy()
df_blast.columns = ['query_id', 'subject_id', '%_Identity', 'alignment_length ', 'mismatches ', 'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e-value', 'bit_score']

df_blast.head()

,query_id,subject_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score
0,YDR425W,XP_962863.3,26.145,524,345,8,122,623,112,615,6.370000e-52,188.0
1,YDR425W,XP_003663337.1,32.869,359,212,6,122,472,97,434,5.970000e-50,182.0
2,YJL036W,XP_003658349.1,34.038,426,234,6,28,418,56,469,1.980000e-86,271.0
3,YJL036W,XP_965318.1,33.333,423,238,6,28,418,69,479,8.400000e-81,256.0
4,YJL036W,XP_011393870.1,23.810,252,146,8,75,299,444,676,1.220000e-08,56.6


In [9]:
df_blast['e-value'] = df_blast['e-value'].apply(lambda x: '{:.10f}'.format(x))

# df_blast改名为Protein_id
df_blast.rename(columns={'subject_id': 'Protein_id'}, inplace=True)

# Protein_id不保留小数点及后面的数字
df_blast['Protein_id'] = df_blast['Protein_id'].apply(lambda x: x.split('.')[0])

df_blast.head()

,query_id,Protein_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score
0,YDR425W,XP_962863,26.145,524,345,8,122,623,112,615,0.0000000000,188.0
1,YDR425W,XP_003663337,32.869,359,212,6,122,472,97,434,0.0000000000,182.0
2,YJL036W,XP_003658349,34.038,426,234,6,28,418,56,469,0.0000000000,271.0
3,YJL036W,XP_965318,33.333,423,238,6,28,418,69,479,0.0000000000,256.0
4,YJL036W,XP_011393870,23.810,252,146,8,75,299,444,676,0.0000000122,56.6


In [10]:
# 读取Mt-Gene_information.tsv
df_mt = pd.read_csv('./Mt-Gene_information.tsv', sep='\t').copy()
df_mt = df_mt[['Protein_id','Gene','Description','Pathway Name','Pathway ID','GO ID','PFAM','Amino_acid_sequence']]

df_mt.head()

,Protein_id,Gene,Description,Pathway Name,Pathway ID,GO ID,PFAM,Amino_acid_sequence
0,XP_003660026,MYCTH_100011,2OG-Fe(II) oxygenase superfamily,NaN,NaN,NaN,2OG-FeII_Oxy_2,MSNITNKVISQTLSPVDVPPKTFTSSDTTKSRLAAQRLRIATKRAA...
1,XP_003662402,MYCTH_100068,Belongs to the glycosyl hydrolase 11 (cellulas...,NaN,NaN,"GO:0005576,GO:0005975,GO:0030248,GO:0004553,GO...","CBM_1,Glyco_hydro_11",MVALSSLLVAASAAAVAVAAPSEALQKRQTLTSSQTGFHDGFYYSF...
2,XP_003662373,MYCTH_100089,"protocatechuate 3,4-dioxygenase activity",NaN,NaN,"GO:0005506,GO:0016702,GO:0055114",Dioxygenase_C,MTLKQLLMSALLAAPALAHPAHKEKVYAHAALPLERKSLAHCAAKF...
3,XP_003662324,MYCTH_100127,YCII-related domain,NaN,NaN,NaN,YCII,MATEAPKKMEWLVVVPDFPGAHEKRIAVRPQHFANLGPVKESGVFQ...
4,XP_003662142,MYCTH_100290,Mis6,NaN,NaN,"GO:0000776,GO:0034508",CENP-I,MPSPEDDDLGGLIGDLEAASKVPAKRRQTNIKSTVEKATALLYDRG...


In [11]:
df_nc = pd.read_csv('./Nc-Gene_information.tsv', sep='\t').copy()
df_nc = df_nc[['Protein_id','Gene','Description','Pathway Name','Pathway ID','GO ID','PFAM','Amino_acid_sequence']]

df_nc.head()

,Protein_id,Gene,Description,Pathway Name,Pathway ID,GO ID,PFAM,Amino_acid_sequence
0,XP_956990,NCU00003,NaN,NaN,NaN,NaN,NaN,MAKSLSDMVELISSPTAVKSLDSDVSPRTQNSSKPKDYFSPQPRAH...
1,XP_956991,NCU00004,Universal stress protein family,NaN,NaN,NaN,Usp,MSSSLTKSSNSAPRFRQHVGFDNLPAGDSTKNNTSSFTLHVKHQGY...
2,XP_956992,NCU00005,Abscisic acid G-protein coupled receptor,NaN,NaN,GO:0016020,"ABA_GPCR,GPHR_N",MSCGVDSCGAPIPTSPIGTLFSLSPFIATFAIVSSVVAGKVFPKLS...
3,XP_011393719,NCU00006,Src homology 3 domains(BUD14),NaN,NaN,GO:0005515,"RA,SH3_1",MTRPEIIRADTIDLQSRGAHSTQGFHSSTSNGPLAPHQAETLREVA...
4,XP_956994,NCU00007,PalH/RIM21(RIM21),NaN,NaN,NaN,PalH,MEPRQLFSDPTADPISTAGAASNALSCASFNLPEGGILQLPNGEII...


In [12]:
"""
如果df_blast中的Protein_id与df_mt中Protein_id是一样的
那么就将df_mt中的Gene、Description、Pathway Name、Pathway ID、GO ID、PFAM、Amino_acid_sequence添加到df_blast中
"""

df_1 = pd.merge(df_blast, df_mt, on='Protein_id', how='left')
df_1

,query_id,Protein_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score,Gene,Description,Pathway Name,Pathway ID,GO ID,PFAM,Amino_acid_sequence
0,YDR425W,XP_962863,26.145,524,345,8,122,623,112,615,0.0000000000,188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,YDR425W,XP_003663337,32.869,359,212,6,122,472,97,434,0.0000000000,182.0,MYCTH_2305149,"PhoX homologous domain, present in p47phox and...",NaN,NaN,GO:0035091,"PX,Vps5",MWNDEDNNPYGGSFERRDSFASSANPSSPITHDYPRYDAPNTPSFT...
2,YJL036W,XP_003658349,34.038,426,234,6,28,418,56,469,0.0000000000,271.0,MYCTH_2293999,Vps5 C terminal like(SNX4),Mt-Endocytosis,mtm04144,GO:0035091,"PX,Vps5",MAVLDQDNFSNISWHSEQNPDAAGPSTSASSHDRSHEARTGDGRHV...
3,YJL036W,XP_965318,33.333,423,238,6,28,418,69,479,0.0000000000,256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YJL036W,XP_011393870,23.810,252,146,8,75,299,444,676,0.0000000122,56.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6784,YPR080W,XP_958508,22.814,469,266,19,10,439,199,610,0.0000000000,77.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6785,YPR080W,XP_003662584,22.979,470,264,19,10,439,191,602,0.0000000001,63.5,MYCTH_2303368,Elongation factor Tu domain 2,NaN,NaN,"GO:0005525,GO:0003924,GO:0005525","GTP_EFTU,GTP_EFTU_D2,GTP_EFTU_D3",MTSKSAKQSSHERRKGEAALSDFAEYVEKQQALRYPSSKQVAATAA...
6786,YPR080W,XP_958122,26.354,277,143,9,9,279,123,344,0.0000000001,63.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6787,YPR080W,XP_962160,27.094,203,106,8,9,200,58,229,0.0000000215,56.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# 如果df_1中的Protein_id与df_nc中Protein_id是一样的
# 那么就将df_nc中的Gene、Description、Pathway Name、Pathway ID、GO ID、PFAM、Amino_acid_sequence添加到df_1中
df_2 = pd.merge(df_1, df_nc, on='Protein_id', how='left')
df_2

,query_id,Protein_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,...,GO ID_x,PFAM_x,Amino_acid_sequence_x,Gene_y,Description_y,Pathway Name_y,Pathway ID_y,GO ID_y,PFAM_y,Amino_acid_sequence_y
0,YDR425W,XP_962863,26.145,524,345,8,122,623,112,615,...,NaN,NaN,NaN,NCU06259,Vps5 C terminal like(ATG20),NaN,NaN,GO:0035091,"PX,Vps5",MWNDEDNNPYGSFERRDSFASSTNPASPTARDYSIDARFPTPQDVR...
1,YDR425W,XP_003663337,32.869,359,212,6,122,472,97,434,...,GO:0035091,"PX,Vps5",MWNDEDNNPYGGSFERRDSFASSANPSSPITHDYPRYDAPNTPSFT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YJL036W,XP_003658349,34.038,426,234,6,28,418,56,469,...,GO:0035091,"PX,Vps5",MAVLDQDNFSNISWHSEQNPDAAGPSTSASSHDRSHEARTGDGRHV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YJL036W,XP_965318,33.333,423,238,6,28,418,69,479,...,NaN,NaN,NaN,NCU03218,Vps5 C terminal like(SNX4),Nc-Endocytosis,ncr04144,GO:0035091,"PX,Vps5",MAVIDQDNFSNISWHSEQNAESAASTAQVHHESNSSPEYARSGPDD...
4,YJL036W,XP_011393870,23.810,252,146,8,75,299,444,676,...,NaN,NaN,NaN,NCU05715,"PhoX homologous domain, present in p47phox and...",NaN,NaN,"GO:0035091,GO:0035091,GO:0042147",PX,MSLFGSSPPNDGSAALNPAKTANSSRSTLFDNEAPTTRSGSALFAD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6784,YPR080W,XP_958508,22.814,469,266,19,10,439,199,610,...,NaN,NaN,NaN,NCU09513,Elongation factor Tu domain 2,NaN,NaN,"GO:0003924,GO:0005525,GO:0005525","GTP_EFTU,GTP_EFTU_D2,GTP_EFTU_D3",MASKQAPHERRKGEAALSDFAEYVERQQALRYPNKAAQSQTATTSA...
6785,YPR080W,XP_003662584,22.979,470,264,19,10,439,191,602,...,"GO:0005525,GO:0003924,GO:0005525","GTP_EFTU,GTP_EFTU_D2,GTP_EFTU_D3",MTSKSAKQSSHERRKGEAALSDFAEYVEKQQALRYPSSKQVAATAA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6786,YPR080W,XP_958122,26.354,277,143,9,9,279,123,344,...,NaN,NaN,NaN,NCU05927,Promotes mitochondrial protein synthesis. May ...,NaN,NaN,"GO:0003924,GO:0005525,GO:0005525","EFG_C,GTP_EFTU,GTP_EFTU_D2,LepA_C",MRGALCRPDVLMRPCLRPCARLPRLPPSRRLPFSITTSCQQLPRTS...
6787,YPR080W,XP_962160,27.094,203,106,8,9,200,58,229,...,NaN,NaN,NaN,NCU07021,P-loop containing nucleoside triphosphate hydr...,NaN,NaN,"GO:0003924,GO:0005525,GO:0005525","EFG_C,EFG_II,GTP_EFTU,GTP_EFTU_D2",MLLAAPLRGCRHGHRAFLPLSLPTVVKSTSPLAATRFPHFKRFYAD...


In [14]:
# 保存df_blast为excel   
df_2.to_excel('比对结果.xlsx', index=False)